In [1]:
import vaex as vx

In [2]:
DATA = './data/nyc-taxi/*.parquet'

In [3]:
%%time

df = vx.open(DATA)

CPU times: user 269 ms, sys: 78.1 ms, total: 347 ms
Wall time: 428 ms


In [4]:
df.column_names

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [5]:
df.shape

(91809949, 19)

In [6]:
df.head()

#,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.2,1,N,238,239,1,6,3,0.5,1.47,0,0.3,11.27,2.5,--
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.2,1,N,239,238,1,7,3,0.5,1.5,0,0.3,12.3,2.5,--
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.6,1,N,238,238,1,6,3,0.5,1,0,0.3,10.8,2.5,--
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.8,1,N,238,151,1,5.5,0.5,0.5,1.36,0,0.3,8.16,0,--
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,4.8,0,--
5,2,2020-01-01 00:09:44,2020-01-01 00:10:37,1,0.03,1,N,7,193,2,2.5,0.5,0.5,0,0,0.3,3.8,0,--
6,2,2020-01-01 00:39:25,2020-01-01 00:39:29,1,0,1,N,193,193,1,2.5,0.5,0.5,0.01,0,0.3,3.81,0,--
7,2,2019-12-18 15:27:49,2019-12-18 15:28:59,1,0,5,N,193,193,1,0.01,0,0,0,0,0.3,2.81,2.5,--
8,2,2019-12-18 15:30:35,2019-12-18 15:31:35,4,0,1,N,193,193,1,2.5,0.5,0.5,0,0,0.3,6.3,2.5,--
9,1,2020-01-01 00:29:01,2020-01-01 00:40:28,2,0.7,1,N,246,48,1,8,3,0.5,2.35,0,0.3,14.15,2.5,--


In [7]:
%%time

df['is_congested'] = df.congestion_surcharge.fillna(0) > 0

df.groupby(
    'DOLocationID',
    agg={
        'mean_fare': vx.agg.mean('fare_amount'),
        'count': vx.agg.count('*'),
        'congestion': vx.agg.mean('is_congested')
    },
    progress=True
).sort('count', ascending=False)

groupby [########################################] 100.00% elapsed time  :     2.77s =  0.0m =  0.0h
 CPU times: user 4.31 s, sys: 727 ms, total: 5.03 s
Wall time: 2.83 s


#,DOLocationID,mean_fare,count,congestion
0,236,9.600090780733105,4109242,0.9663884482831627
1,237,9.118917058205195,3845516,0.97569610944279
2,161,10.634795718163923,3165651,0.9714930673027443
3,170,10.632018730016933,2730697,0.9673478968922586
4,141,9.888821380453033,2567317,0.9705470730727838
...,...,...,...,...
259,2,42.227734375000004,128,0.0546875
260,105,18.675581395348836,43,0.7674418604651163
261,110,80.33333333333333,3,0.0
262,199,0.0,2,0.0


In [8]:
%%time

with vx.progress.tree('rich'):
    df['is_congested'] = df.congestion_surcharge.fillna(0) > 0

    grouped = df.groupby(
        'DOLocationID',
        agg={
            'mean_fare': vx.agg.mean('fare_amount'),
            'count': vx.agg.count('*'),
            'congestion': vx.agg.mean('is_congested')
        },
        progress=True
    ).sort('count', ascending=False)
    
grouped

  vaex                                          ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 00.00s     
  vaex                                          ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 00.00s   

CPU times: user 4.36 s, sys: 684 ms, total: 5.05 s
Wall time: 2.77 s


#,DOLocationID,mean_fare,count,congestion
0,236,9.600090780733096,4109242,0.9663884482831627
1,237,9.118917058205197,3845516,0.97569610944279
2,161,10.634795718163858,3165651,0.9714930673027443
3,170,10.632018730016934,2730697,0.9673478968922586
4,141,9.888821380453013,2567317,0.9705470730727838
...,...,...,...,...
259,2,42.227734375000004,128,0.0546875
260,105,18.675581395348836,43,0.7674418604651163
261,110,80.33333333333333,3,0.0
262,199,0.0,2,0.0
